# Micro Mortgages - Stacking
- Author: Oliver Mueller
- Last update: 26.01.2024

## Initialize notebook
Load required packages. Set up workspace, e.g., set theme for plotting and initialize the random number generator.

In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier


In [ ]:
plt.style.use('fivethirtyeight')

## Problem description

In India, there are about 20 million home loan (mortgage) aspirants
working in the informal sector:

- Monthly income between INR 20,000-25,000 (\$ 325-400)
- Typically no formal accounts and documents (e.g., tax returns, income proofs, bank statements)
- Often use services of money lenders with interest rates between 30 and 60% per annum

Providing mortgages to this group of customers requires to quickly and
efficiently assess their creditworthiness. Due to a lack of formal
documents and objective data, most financial institutions perform
interview-based processes to decide about these loan requests:

Strength of the current process:

-   Interview-based field assessment

-   Relaxation of document requirements

Weaknesses of the current process:

-   Costly (total transaction costs as high as 30% of loan volume)

-   Subjective judgments; depends on individual skills and motivations

-   Low reliability across branches and credit officers

-   Risk of corruption and fraud

## Load data

Load training data from CSV file.

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/olivermueller/vhbprodok_datascience/main/micro_mortgages/data/micromortgage.csv')

In [ ]:
data.head()

## Prepare data

In [ ]:
data = data.drop(['ID'], axis=1)
data["Tier"] = data["Tier"].apply(lambda x: "T"+str(x))

In [ ]:
X = data.drop("Decision", axis=1)
y = data["Decision"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
categorical_features = X_train.select_dtypes(include='object').columns
numerical_features = X_train.select_dtypes(exclude='object').columns

In [ ]:
categorical_features

In [ ]:
numerical_features

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(X_train[categorical_features])

X_train_cat = enc.transform(X_train[categorical_features])
X_test_cat = enc.transform(X_test[categorical_features])

X_train_cat = pd.DataFrame(X_train_cat, columns=enc.get_feature_names_out(categorical_features))
X_test_cat = pd.DataFrame(X_test_cat, columns=enc.get_feature_names_out(categorical_features))

In [ ]:
X_train_cat.head()

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_features]) 

X_train_num = scaler.transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])

X_train_num = pd.DataFrame(X_train_num, columns=numerical_features)
X_test_num = pd.DataFrame(X_test_num, columns=numerical_features)

In [ ]:
X_train_num.head()

In [ ]:
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

## LASSO

In [ ]:
model_lasso_cv = lambda: GridSearchCV(
                estimator=LogisticRegression(random_state=42),
                param_grid={
                        'C': np.logspace(-4, 4, 10)
                    }, cv=5, n_jobs=-1
                )

In [ ]:
model_lasso_tuned = model_lasso_cv()
model_lasso_tuned.fit(X_train, y_train)

In [ ]:
pred_lasso_label = model_lasso_tuned.predict(X_test)
pred_lasso_proba = model_lasso_tuned.predict_proba(X_test)[:,1]

In [ ]:
print(classification_report(y_test, pred_lasso_label))

In [ ]:
roc_auc_score(y_test, pred_lasso_proba)

## Random Forest

In [ ]:
model_rf_cv = lambda: GridSearchCV(
                estimator=RandomForestClassifier(random_state=42),
                param_grid={
                        'n_estimators': [100, 200, 300],
                        'max_depth': [3, 5, None],
                        'min_samples_leaf': [2, 5, 10]
                    }, cv=5, n_jobs=-1
                )

In [ ]:
model_rf_tuned = model_rf_cv()
model_rf_tuned.fit(X_train, y_train)

In [ ]:
pred_rf_label = model_rf_tuned.predict(X_test)
pred_rf_proba = model_rf_tuned.predict_proba(X_test)[:,1]

In [ ]:
print(classification_report(y_test, pred_rf_label))

In [ ]:
roc_auc_score(y_test, pred_rf_proba)

## Stack models

Here, we will use a simple stacking approach by combining the predictions of the LASSO and Random Forest models with equal weight.

In [ ]:
pred_final_proba = (pred_lasso_proba + pred_rf_proba) / 2

In [ ]:
roc_auc_score(y_test, pred_final_proba)